# Nahuatl Notebook for the WHP_EarlyNahuatl_Dataset

This notebook processes Nahuatl dictionary data, analyzing HTML tags, repairing malformed tags, and extracting citations and cross-references. This is a merged version of Todd's version and I where there is a SQLite-based data management approach.

## Setup and Imports

In [1]:
import pandas as pd
import re
import os
import numpy as np
import hashlib
import glob
import csv
import sqlite3
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional, Union
from inscriptis import get_text
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime

In [2]:
# Create working directory
os.makedirs('working_files', exist_ok=True)

# load in the SQLite database holding the WHP Dataset
conn = sqlite3.connect('../../data/sqLiteDb/Whp_Raw_Dataset.db')
table_name = "WHP_EarlyNahuatl_Data"

tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)
tables


# If there's issues check the following
# Possible solutions:
# 1. Ensure the db file is in the correct directory
# 2. Check the exact filename
# 3. Verify the file extension

,name
0,WHP_EarlyNahuatl_Data


In [3]:
def save_to_excel(data_dict: Dict[str, pd.DataFrame], filename: str, directory: str = 'working_files'):
    """Save multiple DataFrames as sheets in an Excel file"""
    filepath = os.path.join(directory, filename)
    with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
        for sheet_name, df in data_dict.items():
            # Truncate sheet name if too long (Excel limit is 31 characters)
            clean_sheet_name = sheet_name[:31] if len(sheet_name) > 31 else sheet_name
            df.to_excel(writer, sheet_name=clean_sheet_name, index=False)
    print(f"Saved to: {filepath}")

In [4]:
def save_dataframe(df: pd.DataFrame, filename: str, directory: str = 'working_files'):
    """Save a single DataFrame to CSV"""
    filepath = os.path.join(directory, filename)
    df.to_csv(filepath, index=False)
    print(f"Saved to: {filepath}")

In [5]:
def save_to_sqlite(df: pd.DataFrame, table_name: str, conn: sqlite3.Connection, if_exists: str = 'replace'):
    """Save DataFrame to SQLite table"""
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)
    print(f"Saved to SQLite table: {table_name}")

## Step 1: Import Data and Create Working Copy

In [6]:
def load_data_from_csv(filename: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load data and create a working copy"""
    print(f"Loading data from: {filename}")

    # Read the original data
    original_df = pd.read_csv(filename)

    # Create working copy
    working_df = original_df.copy()

    print(f"Data loaded successfully:")
    print(f"- Shape: {original_df.shape}")
    print(f"- Columns: {list(original_df.columns)}")

    return original_df, working_df

In [7]:
def load_data_from_sqlite(db_path: str, table_name: str = "WHP_EarlyNahuatl_Data") -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load data from SQLite and create a working copy"""
    print(f"Loading data from: {db_path}")
    
    conn = sqlite3.connect(db_path)
    original_df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    working_df = original_df.copy()
    
    print(f"Data loaded successfully:")
    print(f"- Shape: {original_df.shape}")
    print(f"- Columns: {list(original_df.columns)}")
    
    # Don't close connection yet - return it for later use
    return original_df, working_df, conn

In [8]:
# Load your data

original_df = pd.read_sql("SELECT * FROM WHP_EarlyNahuatl_Data", conn)
df = original_df.copy(deep=True)

query = "SELECT * FROM WHP_EarlyNahuatl_Data LIMIT 3;"
whp_dataset = pd.read_sql(query, conn)
display(whp_dataset)

cursor = conn.execute(f"PRAGMA table_info({table_name})")
columns_info = cursor.fetchall()
column_names = [col[1] for col in columns_info]

print(column_names)

,Ref,Headword,Orthographic Variants,Principal English Translation,Attestations from sources in English,Attestations from sources in Spanish,Alonso de Molina,Frances Karttunen,Horacio Carochi / English,Andrés de Olmos,Lockhart’s Nahuatl as Written,themes,Spanish Loanword
0,WHP-171879,acazomo.,"accaçomo, acaçomo",<p>perhaps not (adverb) (see Molina)</p>,<p>acaçomo iuhqui yez yn anoço yuhquiez = whet...,None,<p>Acaçomo. quiça no. Aduerbio.<br /> <bibl> A...,<p>AHCAZOMŌ perhaps not / quizá no (M). In on...,<p>àcaçomō = perhaps not<br /> <bibl>Horacio C...,None,None,None,No
1,WHP-171881,ayac.,aiaac,"<p>no one; nobody; or, for someone to be absen...",<p>aiaac mic in mexica = None of the Mexica di...,<p>ayac guincuiliz = no se la quite nadie (Tla...,"<p>Ayac. ninguno, o nadie o estar alguno ausen...","<p>AYĀC no one / ninguno, o nadie (M) See AH-,...","<p>ayāc = no one<br /> <bibl>Horacio Carochi, ...",None,"<p>no one; nobody; or, for someone to be absen...",None,No
2,WHP-171882,acan.,None,"<p>nowhere, no place (see Molina, Karttunen, L...",None,None,<p>acan. en ninguna parte o lugar. aduerbio.<b...,<p>AHCĀN nowhere / en ninguna parte o lugar (M...,"<p>àcān = nowhere<br /> <bibl>Horacio Carochi,...","<p>en ningun lugar, por, de, etc.<br /> <bibl>...",<p>ahcān = (particle) nowhere<br /> <bibl>Jame...,"Cardinal Directions, Cosmos",No


['Ref', 'Headword', 'Orthographic Variants', 'Principal English Translation', 'Attestations from sources in English', 'Attestations from sources in Spanish', 'Alonso de Molina', 'Frances Karttunen', 'Horacio Carochi / English', 'Andrés de Olmos', 'Lockhart’s Nahuatl as Written', 'themes', 'Spanish Loanword']


## Step 2: Save Intermediate Stages

In [9]:
def save_intermediate_stage(df: pd.DataFrame, stage_name: str):
    """Save intermediate processing stage"""
    filename = f"{stage_name}_stage.csv"
    save_dataframe(df, filename)
    return df

def save_intermediate_stage_sqlite(df: pd.DataFrame, stage_name: str, conn: sqlite3.Connection):
    """Save intermediate processing stage to SQLite"""
    table_name = f"{stage_name}_stage"
    save_to_sqlite(df, table_name, conn)
    return df

In [10]:
# Save initial stage
# save_intermediate_stage_sqlite(df, "01_initial", conn)

## Step 3: HTML Tag Analysis

In [11]:
class HTMLTagAnalyzer:
    def __init__(self):
        # HTML tags
        self.html_tags = {
            'p', 'br', 'div', 'span', 'a', 'b', 'i', 'u', 'strong', 'em',
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'table',
            'tr', 'td', 'th', 'img', 'link', 'meta', 'head', 'body', 'html',
            'bibl', 'title', 'sup', 'sub', 'del'
        }
        
        # Define columns that should contain HTML content
        self.content_columns = [
            'Principal English Translation',
            'Attestations from sources in English',
            'Attestations from sources in Spanish',
            'Alonso de Molina',
            'Frances Karttunen', 
            'Horacio Carochi / English',
            'Andrés de Olmos',
            "Lockhart's Nahuatl as Written",
            'Full Original Entry'
        ]
        
        # Known malformed patterns to fix
        self.malformed_patterns = {
            r'</p</bibl>': '</p></bibl>',
            r'<bibl<': '<bibl>',
            r'</bibbl>': '</bibl>',
            r'<bibbl>': '<bibl>',
            r'<bobl>': '<bibl>',
            r'</bobl>': '</bibl>',
            r'<b9bl>': '<bibl>',
            r'<bibi>': '<bibl>',
            r'<bibl></p>': '</bibl></p>',
        }
    
    def detect_malformed_tags(self, text: str) -> List[tuple]:
        """Detect specific malformed tag patterns"""
        if pd.isna(text) or text == '':
            return []
        
        malformed_found = []
        text_str = str(text)
        
        # Check for known malformed patterns
        for pattern, replacement in self.malformed_patterns.items():
            if re.search(pattern, text_str):
                malformed_found.append((pattern, replacement))
        
        # Define self-closing tags that shouldn't be counted in pair matching
        self_closing_tags = {'br', 'hr', 'img', 'input', 'meta', 'link'}
        
        # Better tag counting using regex
        for tag_name in self.html_tags:
            if tag_name in self_closing_tags:
                continue  # Skip self-closing tags
            
            # Use regex to properly count opening tags (with or without attributes)
            # Matches <tag> or <tag attr="...">~
            open_pattern = f"<{tag_name}(?:\\s+[^>]*)?>"
            close_pattern = f'</{tag_name}>'
            
            open_count = len(re.findall(open_pattern, text_str, re.IGNORECASE))
            close_count = len(re.findall(close_pattern, text_str, re.IGNORECASE))
            
            if open_count != close_count:
                malformed_found.append((f'<{tag_name}>', f'Mismatch: {open_count} open, {close_count} closed'))
        return malformed_found
    
    def find_html_tags(self, text: str) -> List[str]:
        """Find all HTML-like tags in text with better handling of malformed tags"""
        if pd.isna(text) or text == '':
            return []
        
        # First fix known malformed patterns
        text_str = str(text)
        for pattern, replacement in self.malformed_patterns.items():
            text_str = re.sub(pattern, replacement, text_str)
        
        # Then find tags
        pattern = r'</?[^<>]+/?>'
        matches = re.findall(pattern, text_str)
        return matches
    
    def analyze_html_tags_in_dataframe(self, df: pd.DataFrame, 
                                      columns_to_check: List[str] = None) -> Dict[str, pd.DataFrame]:
        """Analyze HTML tags only in relevant columns"""
        results = {
            'tag_by_row': [],
            'tag_summary': [],
            'malformed_tags': []
        }
        
        # Use specified columns or default to content columns
        if columns_to_check is None:
            columns_to_check = [col for col in self.content_columns if col in df.columns]
        
        # Track tags by row - only in relevant columns
        for idx, row in df.iterrows():
            for col in columns_to_check:
                if col not in df.columns:
                    continue
                    
                cell_value = row[col]
                if pd.notna(cell_value) and cell_value != '':
                    # Check for malformed tags first
                    malformed = self.detect_malformed_tags(cell_value)
                    if malformed:
                        for pattern, fix in malformed:
                            results['malformed_tags'].append({
                                'Row': idx,
                                'Column': col,
                                'Pattern': pattern,
                                'Suggested_Fix': fix,
                                'Context': str(cell_value)[:100] + '...' if len(str(cell_value)) > 100 else str(cell_value)
                            })
                    
                    # Find tags
                    tags = self.find_html_tags(cell_value)
                    for tag in tags:
                        is_valid = self.is_valid_html_tag(tag)
                        context = self.get_tag_context(cell_value, tag)
                        results['tag_by_row'].append({
                            'Row': idx,
                            'Column': col,
                            'Tag': tag,
                            'Is_Valid_HTML': is_valid,
                            'Context': context
                        })
        
        # Create summaries
        if results['tag_by_row']:
            tag_by_row_df = pd.DataFrame(results['tag_by_row'])
            
            # Tag summary
            tag_counts = Counter([item['Tag'] for item in results['tag_by_row']])
            tag_locations = defaultdict(list)
            
            for item in results['tag_by_row']:
                tag_locations[item['Tag']].append(f"Row {item['Row']}, Col {item['Column']}")
            
            for tag, count in tag_counts.items():
                first_occurrence = next(item for item in results['tag_by_row'] if item['Tag'] == tag)
                results['tag_summary'].append({
                    'Tag': tag,
                    'Count': count,
                    'Is_Valid_HTML': first_occurrence['Is_Valid_HTML'],
                    'Locations': '; '.join(tag_locations[tag][:5]) + ('...' if len(tag_locations[tag]) > 5 else ''),
                    'Sample_Context': first_occurrence['Context']
                })
            
            tag_summary_df = pd.DataFrame(results['tag_summary']).sort_values('Count', ascending=False)
        else:
            tag_by_row_df = pd.DataFrame()
            tag_summary_df = pd.DataFrame()
        
        malformed_df = pd.DataFrame(results['malformed_tags']) if results['malformed_tags'] else pd.DataFrame()
        
        return {
            'HTML_Tags_by_Row': tag_by_row_df,
            'HTML_Tags_Summary': tag_summary_df,
            'Malformed_Tags': malformed_df
        }
    
    def is_valid_html_tag(self, tag: str) -> bool:
        """Check if a tag is a valid HTML tag with better error handling"""
        try:
            # Handle malformed tags better
            if '<//' in tag or '><' in tag:  # Clearly malformed
                return False
            
            # Remove < > and any attributes, get just the tag name
            clean_tag = re.sub(r'^</?([^>\s/]+).*>$', r'\1', tag).lower()
            
            # Additional check for malformed tags
            if '/' in clean_tag or '<' in clean_tag or '>' in clean_tag:
                return False
                
            return clean_tag in self.html_tags
        except:
            return False
    
    def get_tag_context(self, text: str, tag: str, context_chars: int = 50) -> str:
        """Get context around a tag occurrence"""
        if pd.isna(text) or text == '':
            return ''
        
        text_str = str(text)
        tag_pos = text_str.find(tag)
        if tag_pos == -1:
            return ''
        
        start = max(0, tag_pos - context_chars)
        end = min(len(text_str), tag_pos + len(tag) + context_chars)
        context = text_str[start:end]
        
        # Mark the tag in the context
        tag_in_context = context.replace(tag, f"[[[{tag}]]]")
        return tag_in_context

In [12]:
html_analyzer = HTMLTagAnalyzer()
html_results = html_analyzer.analyze_html_tags_in_dataframe(df)

html_results['HTML_Tags_Summary'].to_sql('html_tag_analysis', conn, if_exists='replace', index=False)
html_results['Malformed_Tags'].to_sql('malformed_tags', conn, if_exists='replace', index=False)

# save_intermediate_stage_sqlite(df, "02_htmltag_analysis", conn)
save_to_excel(html_results, "02_html_tag_analysis.xlsx")

Saved to: working_files\02_html_tag_analysis.xlsx


## Step 4: Malformed Tag Detection and Repair

In [22]:
class MalformedTagRepairer:
    def __init__(self):
        self.html_analyzer = HTMLTagAnalyzer()
        
        # Specific patterns for actual HTML malformations
        self.html_malformation_patterns = {
            # Bibliography tag typos
            r'</bibbl>': '</bibl>',
            r'<bibbl>': '<bibl>',
            r'<\?bibl>': '<bibl>',
            r'<bibl<': '<bibl>',
            r'</bobl>': '</bibl>',
            r'<bobl>': '<bibl>',
            r'<b9bl>': '<bibl>',
            r'<bibi>': '<bibl>',
            
            # Paragraph tag issues
            r'</p</bibl>': '</p></bibl>',
            r'<p<': '<p>',
            r'</p>p>': '</p>',
            
            # Structural issues
            r'<<(\w+)>': r'<\1>',        # Double opening: <<bibl> → <bibl>
            r'<(\w+)>>': r'<\1>',        # Double closing: <bibl>> → <bibl>
            r'<(\w+)\s+<': r'<\1>',      # Unclosed with new tag: <bibl <p> → <bibl>
            
            # Common typos
            r'<stron>': '<strong>',
            r'</stron>': '</strong>',
            r'<em >': '<em>',
            r'< (\w+)>': r'<\1>',        # Space after bracket: < bibl> → <bibl>
        }
        
        # HTML tag patterns that indicate this IS supposed to be HTML
        self.html_indicators = [
            r'^</?(?:bibl|p|strong|em|b|i|u|sup|sub|div|span|a|br|h[1-6])[\s>]',
            r'</\w+>$',                  # Closing tags
            r'<\w+\s+\w+="[^"]*"',      # Tags with attributes
        ]

    def is_definitely_non_html(self, tag: str) -> bool:
        """Conservative check - is this definitely NOT HTML?"""
        for pattern in self.non_html_patterns:
            if re.search(pattern, tag):
                return True
        return False

    def is_close_to_html(self, tag: str) -> bool:
        """MUCH more conservative - only obvious HTML-like malformations"""
        # If it's definitely non-HTML content, don't even consider it
        if self.is_definitely_non_html(tag):
            return False
        
        # Only consider malformed if it's in our exact repair list
        if tag in self.exact_repairs:
            return True
            
        # Or if it has obvious HTML tag structure problems
        malformed_patterns = [
            r'<[^>]*</[^>]*>',  # Mixed opening/closing in one tag
            r'</[^>]*<[^>]*>',  # Reversed brackets  
            r'<[^>]*<[^>]*>',   # Double opening brackets
        ]
        
        for pattern in malformed_patterns:
            if re.search(pattern, tag):
                return True
                
        return False
    
    def is_likely_malformed_html(self, tag: str) -> bool:
        """Check if this tag looks like malformed HTML (not just non-HTML content)"""
        tag_lower = tag.lower()
        
        # Check if it matches any HTML indicators
        for pattern in self.html_indicators:
            if re.search(pattern, tag_lower):
                return True
        
        # Check against known malformation patterns
        for pattern in self.html_malformation_patterns.keys():
            if re.search(pattern, tag):
                return True
                
        return False
    
    

    def find_malformed_tags(self, text: str) -> List[str]:
        """Find only actual malformed HTML tags"""
        if pd.isna(text) or text == '':
            return []

        malformed_tags = []
        
        # Direct pattern matching for known malformations
        for pattern, replacement in self.html_malformation_patterns.items():
            matches = re.findall(pattern, str(text))
            malformed_tags.extend(matches)
        
        # Additional check for malformed structure
        all_brackets = re.findall(r'<[^<>]*>', str(text))
        for tag in all_brackets:
            if (not self.html_analyzer.is_valid_html_tag(tag) and 
                self.is_likely_malformed_html(tag) and
                tag not in malformed_tags):  # Avoid duplicates
                malformed_tags.append(tag)
        
        return malformed_tags

    def is_close_to_html(self, tag: str) -> bool:
        """Check if a malformed tag is close to valid HTML"""
        malformed_patterns = [
            r'<[^>]*</[^>]*>',  # Mixed opening/closing
            r'</[^>]*<[^>]*>',  # Reversed brackets
            r'<[^>]*<[^>]*>',   # Double opening
            r'<[^/>][^>]*[^/]>$', # Missing closing slash or improper format
        ]

        for pattern in malformed_patterns:
            if re.search(pattern, tag):
                return True
        return False

    def suggest_repair(self, tag: str) -> str:
        """Suggest repair for malformed HTML tags"""
        tag = tag.strip()
        
        # Check direct pattern matches first
        for pattern, replacement in self.html_malformation_patterns.items():
            if re.search(pattern, tag):
                return re.sub(pattern, replacement, tag)
        
        # Additional repair logic
        tag_lower = tag.lower()
        
        # Fix common spacing issues
        if re.match(r'^<\s+\w+', tag):
            return re.sub(r'^<\s+', '<', tag)
        if re.match(r'^<\w+\s+>', tag):
            return re.sub(r'\s+>$', '>', tag)
            
        # Return unchanged if no clear repair
        return tag
    def analyze_malformed_tags(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """Analyze only actual malformed HTML tags across DataFrame"""
        results = {
            'malformed_by_row': [],
            'malformed_summary': []
        }

        print("Scanning for actual HTML malformations...")
        
        for idx, row in df.iterrows():
            for col in df.columns:
                cell_value = row[col]
                if pd.notna(cell_value) and cell_value != '':
                    malformed_tags = self.find_malformed_tags(cell_value)
                    for tag in malformed_tags:
                        context = self.html_analyzer.get_tag_context(cell_value, tag)
                        suggested_repair = self.suggest_repair(tag)
                        results['malformed_by_row'].append({
                            'Row': idx,
                            'Column': col,
                            'Malformed_Tag': tag,
                            'Suggested_Repair': suggested_repair,
                            'Context': context
                        })

        # Create summary
        tag_counts = Counter([item['Malformed_Tag'] for item in results['malformed_by_row']])
        tag_locations = defaultdict(list)

        for item in results['malformed_by_row']:
            tag_locations[item['Malformed_Tag']].append(f"Row {item['Row']}, Col {item['Column']}")

        for tag, count in tag_counts.items():
            first_occurrence = next(item for item in results['malformed_by_row'] if item['Malformed_Tag'] == tag)
            results['malformed_summary'].append({
                'Malformed_Tag': tag,
                'Count': count,
                'Suggested_Repair': first_occurrence['Suggested_Repair'],
                'Locations': '; '.join(tag_locations[tag][:5]) + ('...' if len(tag_locations[tag]) > 5 else ''),
                'Sample_Context': first_occurrence['Context']
            })

        malformed_by_row_df = pd.DataFrame(results['malformed_by_row'])
        malformed_summary_df = pd.DataFrame(results['malformed_summary']).sort_values('Count', ascending=False)

        print(f"Found {len(malformed_by_row_df)} actual HTML malformations")
        
        return {
            'Malformed_Tags_by_Row': malformed_by_row_df,
            'Malformed_Tags_Summary': malformed_summary_df
        }

    def repair_tags(self, df: pd.DataFrame, tag_to_repair: str, replacement: str,
               scope: str = 'global', specific_column: str = None,
               specific_row: int = None) -> pd.DataFrame:
        """Repair malformed tags in DataFrame"""
        df_repaired = df.copy()
        
        print("." * 40)
        print(f"Repairing: '{tag_to_repair}' → '{replacement}'")
        print(">" * 40)
        
        if scope == 'global':
            replacements_made = 0
            for col in df_repaired.columns:
                # Count before
                before_count = df_repaired[col].astype(str).str.contains(
                    re.escape(tag_to_repair), regex=True
                ).sum()
                
                # Make replacement - DON'T escape the replacement
                df_repaired[col] = df_repaired[col].astype(str).str.replace(
                    tag_to_repair, replacement, regex=False  # Use literal replacement
                )
                
                # Count after
                after_count = df_repaired[col].astype(str).str.contains(
                    re.escape(tag_to_repair), regex=True
                ).sum()
                
                column_replacements = before_count - after_count
                if column_replacements > 0:
                    print(f"\tColumn '{col}': {column_replacements} replacements")
                    # Show sample context with actual replacement
                    sample_rows = df_repaired[col].astype(str).str.contains(
                        re.escape(replacement), regex=True
                    )
                    if sample_rows.any():
                        sample_idx = sample_rows.idxmax()
                        context = self.html_analyzer.get_tag_context(
                            df_repaired.loc[sample_idx, col], replacement
                        )
                        print(f"\t\tSample context: {context[:100]}...")
                
                replacements_made += column_replacements
                
            print(f"Total replacements made: {replacements_made}")
            
        elif scope == 'column' and specific_column:
            if specific_column in df_repaired.columns:
                df_repaired[specific_column] = df_repaired[specific_column].astype(str).str.replace(
                    tag_to_repair, replacement, regex=False
                )
        elif scope == 'row' and specific_row is not None:
            for col in df_repaired.columns:
                if pd.notna(df_repaired.loc[specific_row, col]):
                    cell_value = str(df_repaired.loc[specific_row, col])
                    df_repaired.loc[specific_row, col] = cell_value.replace(tag_to_repair, replacement)
        elif scope == 'cell' and specific_column and specific_row is not None:
            if specific_column in df_repaired.columns and pd.notna(df_repaired.loc[specific_row, specific_column]):
                cell_value = str(df_repaired.loc[specific_row, specific_column])
                df_repaired.loc[specific_row, specific_column] = cell_value.replace(tag_to_repair, replacement)
        
        return df_repaired

In [23]:
# Step 4: Malformed Tag Detection and Repair
print("Step 4: Analyzing malformed tags...")

# Use the enhanced MalformedTagRepairer class
malformed_repairer = MalformedTagRepairer()
malformed_results = malformed_repairer.analyze_malformed_tags(df)

print("Malformed Tags Summary:")
if not malformed_results['Malformed_Tags_Summary'].empty:
    display(malformed_results['Malformed_Tags_Summary'])
    
    # Save to SQLite
    malformed_results['Malformed_Tags_Summary'].to_sql(
        'malformed_tags_summary', conn, if_exists='replace', index=False
    )
    malformed_results['Malformed_Tags_by_Row'].to_sql(
        'malformed_tags_by_row', conn, if_exists='replace', index=False
    )
    
    # Apply repairs automatically
    print("\nApplying repairs...")
    for _, row in malformed_results['Malformed_Tags_Summary'].iterrows():  # Fixed syntax
        malformed_tag = row['Malformed_Tag']
        suggested_repair = row['Suggested_Repair']
        if malformed_tag != suggested_repair:
            print(f"Repairing '{malformed_tag}' -> '{suggested_repair}'")
            df = malformed_repairer.repair_tags(
                df, malformed_tag, suggested_repair, scope='global'
            )
        else:
            print(f"Tag '{malformed_tag}' needs no repair")
    
    print(f"✓ Applied {len(malformed_results['Malformed_Tags_Summary'])} HTML repairs")
    
else:
    print("No malformed HTML tags found!")

# Save to Excel and SQLite
save_to_excel(malformed_results, "04_malformed_tag_analysis.xlsx")
save_intermediate_stage_sqlite(df, "04_malformed_repair", conn)

print("Step 4 complete: HTML malformation detection and repair")

Step 4: Analyzing malformed tags...
Scanning for actual HTML malformations...
Found 16 actual HTML malformations
Malformed Tags Summary:


,Malformed_Tag,Count,Suggested_Repair,Locations,Sample_Context
4,<bibbl>,7,<bibl>,"Row 18189, Col Attestations from sources in En...","e soil. (central Mexico, sixteenth century)<br..."
0,<bibl<,2,<bibl>,"Row 2927, Col Attestations from sources in Spa...","fibras vegetales para transportar granos.""<br..."
1,em,1,em,"Row 2927, Col Attestations from sources in Spa...","s vegetales para transportar granos.""<br /> <b..."
2,</wup>,1,</wup>,"Row 3616, Col Attestations from sources in Eng...",l></p> <p>ynic omochiuh missas 10 <strong>p<wu...
3,<bibi>,1,<bibl>,"Row 16873, Col Attestations from sources in En...","te: a ""tonsured"" priest had a shaved head.]<br..."
5,bibl,1,bibl,"Row 19677, Col Attestations from sources in En...","ished. (central Mexico, sixteenth century)<br ..."
6,<b9bl>,1,<bibl>,"Row 30486, Col Attestations from sources in En...",son of a Xonacatl who had died around 1530.<br...
7,<bobl>,1,<bibl>,"Row 30849, Col Alonso de Molina",uixti.) enxaguar la ropa despues de lauada.<br...
8,</bobl>,1,</bibl>,"Row 31700, Col Attestations from sources in Sp...",blicaciones/publicadigital/libros/cuentos_i......



Applying repairs...
Repairing '<bibbl>' -> '<bibl>'
........................................
Repairing: '<bibbl>' → '<bibl>'
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
	Column 'Attestations from sources in English': 7 replacements
		Sample context: ne or not (Coyoacan, mid-sixteenth century)<br /> [[[<bibl>]]]Beyond the Codices, eds. Arthur J.O. A...
Total replacements made: 7
Repairing '<bibl<' -> '<bibl>'
........................................
Repairing: '<bibl<' → '<bibl>'
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
	Column 'Principal English Translation': 1 replacements
		Sample context: <p>to do or make all; completely</p> <p>[[[<bibl>]]]Robert Haskett and Stephanie Wood's notes from N...
	Column 'Attestations from sources in Spanish': 1 replacements
		Sample context: liz = no se la quite nadie (Tlaxcala, 1609)<br /> [[[<bibl>]]]Vidas y bienes olvidados: Testamentos ...
Total replacements made: 2
Tag 'em' needs no repair
Tag '</wup>' needs no repair
Repairing '<bibi>' -> '<bibl>'
..

## Step 5: Non-HTML Tag Detection

In [24]:
class NonHTMLTagProcessor:
    def __init__(self):
        self.html_analyzer = HTMLTagAnalyzer()
        
        # Known HTML tags that should never be encoded
        self.valid_html_tags = {
            'p', 'br', 'div', 'span', 'a', 'b', 'i', 'u', 'strong', 'em',
            'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'ul', 'ol', 'li', 'table',
            'tr', 'td', 'th', 'img', 'link', 'meta', 'head', 'body', 'html',
            'bibl', 'title', 'sup', 'sub', 'del'
        }

    def get_tag_name(self, tag: str) -> str:
        """Extract just the tag name from a tag"""
        match = re.match(r'^</?([a-zA-Z][a-zA-Z0-9]*)', tag)
        if match:
            return match.group(1).lower()
        return ""

    def find_non_html_tags(self, text: str) -> List[str]:
        """Find tags that are definitely not HTML"""
        if pd.isna(text) or text == '':
            return []

        # Find all < ... > patterns
        pattern = r'<[^<>]*>'
        all_brackets = re.findall(pattern, str(text))

        non_html_tags = []
        for tag in all_brackets:
            # Skip if it's valid HTML
            if self.html_analyzer.is_valid_html_tag(tag):
                continue
                
            # Skip if it looks like malformed HTML (should have been caught in Step 4)
            if self._looks_like_malformed_html(tag):
                continue
                
            # This must be non-HTML content (linguistic, descriptive, etc.)
            non_html_tags.append(tag)

        return non_html_tags

    def _looks_like_malformed_html(self, tag: str) -> bool:
        """Check if tag looks like malformed HTML (should have been caught in Step 4)"""
        tag_name = self.get_tag_name(tag)
        
        # If it has a recognizable HTML tag name, it's likely malformed HTML
        if tag_name in self.valid_html_tags:
            return True
            
        # Check for HTML-like patterns
        html_like_patterns = [
            r'^</?[a-zA-Z][a-zA-Z0-9]*\s',    # Tag with attributes
            r'bibl', r'href', r'www', r'http', # HTML-related content
        ]
        
        for pattern in html_like_patterns:
            if re.search(pattern, tag, re.IGNORECASE):
                return True
        return False

    def analyze_non_html_tags(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """Analyze non-HTML tags across DataFrame"""
        results = {
            'non_html_by_row': [],
            'non_html_summary': []
        }

        print("Scanning for non-HTML content in angle brackets...")

        for idx, row in df.iterrows():
            for col in df.columns:
                cell_value = row[col]
                if pd.notna(cell_value) and cell_value != '':
                    non_html_tags = self.find_non_html_tags(cell_value)
                    for tag in non_html_tags:
                        context = self.html_analyzer.get_tag_context(cell_value, tag)
                        results['non_html_by_row'].append({
                            'Row': idx,
                            'Column': col,
                            'Non_HTML_Tag': tag,
                            'Context': context
                        })

        # Create summary
        tag_counts = Counter([item['Non_HTML_Tag'] for item in results['non_html_by_row']])
        tag_locations = defaultdict(list)

        for item in results['non_html_by_row']:
            tag_locations[item['Non_HTML_Tag']].append(f"Row {item['Row']}, Col {item['Column']}")

        for tag, count in tag_counts.items():
            first_occurrence = next(item for item in results['non_html_by_row'] if item['Non_HTML_Tag'] == tag)
            results['non_html_summary'].append({
                'Non_HTML_Tag': tag,
                'Count': count,
                'Locations': '; '.join(tag_locations[tag][:5]) + ('...' if len(tag_locations[tag]) > 5 else ''),
                'Sample_Context': first_occurrence['Context']
            })

        # Convert to DataFrames
        non_html_by_row_df = pd.DataFrame(results['non_html_by_row'])
        non_html_summary_df = pd.DataFrame(results['non_html_summary']).sort_values('Count', ascending=False)

        print(f"Found {len(non_html_by_row_df)} non-HTML tags for encoding")

        return {
            'Non_HTML_Tags_by_Row': non_html_by_row_df,
            'Non_HTML_Tags_Summary': non_html_summary_df
        }

    def encode_brackets(self, df: pd.DataFrame, tag_to_encode: str,
                       left_replacement: str = '&lt;', right_replacement: str = '&gt;',
                       scope: str = 'global', specific_column: str = None,
                       specific_row: int = None) -> pd.DataFrame:
        """Encode < > brackets in non-HTML tags"""
        df_encoded = df.copy()

        # Create the encoded replacement
        encoded_tag = tag_to_encode.replace('<', left_replacement).replace('>', right_replacement)
        
        if scope == 'global':
            encodings_made = 0
            for col in df_encoded.columns:
                # Count before
                before_count = df_encoded[col].astype(str).str.contains(
                    re.escape(tag_to_encode), regex=True
                ).sum()
                
                # Make replacement
                df_encoded[col] = df_encoded[col].astype(str).str.replace(
                    tag_to_encode, encoded_tag, regex=False
                )
                
                # Count after
                after_count = df_encoded[col].astype(str).str.contains(
                    re.escape(tag_to_encode), regex=True
                ).sum()
                
                column_encodings = before_count - after_count
                if column_encodings > 0:
                    print(f"  Column '{col}': {column_encodings} encodings")
                encodings_made += column_encodings
            
            print(f"Total encodings made: {encodings_made}")
            
        elif scope == 'column' and specific_column:
            if specific_column in df_encoded.columns:
                df_encoded[specific_column] = df_encoded[specific_column].astype(str).str.replace(
                    tag_to_encode, encoded_tag, regex=False
                )
        elif scope == 'row' and specific_row is not None:
            for col in df_encoded.columns:
                if pd.notna(df_encoded.loc[specific_row, col]):
                    cell_value = str(df_encoded.loc[specific_row, col])
                    df_encoded.loc[specific_row, col] = cell_value.replace(tag_to_encode, encoded_tag)
        elif scope == 'cell' and specific_column and specific_row is not None:
            if specific_column in df_encoded.columns and pd.notna(df_encoded.loc[specific_row, specific_column]):
                cell_value = str(df_encoded.loc[specific_row, specific_column])
                df_encoded.loc[specific_row, specific_column] = cell_value.replace(tag_to_encode, encoded_tag)

        # Handle NaN and empty string conversion
        for col in df_encoded.columns:
            mask_nan = df_encoded[col] == 'nan'
            mask_empty = df_encoded[col] == ''
            df_encoded.loc[mask_nan, col] = np.nan
            df_encoded.loc[mask_empty, col] = ''

        return df_encoded

In [25]:
# Step 5: Non-HTML Tag Detection and Encoding
print("Step 5: Analyzing non-HTML tags...")

non_html_processor = NonHTMLTagProcessor()
non_html_results = non_html_processor.analyze_non_html_tags(df)

print("Non-HTML Tags Summary:")
if not non_html_results['Non_HTML_Tags_Summary'].empty:
    display(non_html_results['Non_HTML_Tags_Summary'])
    
    # Save to SQLite
    non_html_results['Non_HTML_Tags_Summary'].to_sql(
        'non_html_tags_summary', conn, if_exists='replace', index=False
    )
    non_html_results['Non_HTML_Tags_by_Row'].to_sql(
        'non_html_tags_by_row', conn, if_exists='replace', index=False
    )
    
    # Apply encoding automatically
    print("\nApplying bracket encoding...")
    for _, row in non_html_results['Non_HTML_Tags_Summary'].iterrows():
        non_html_tag = row['Non_HTML_Tag']
        print(f"Encoding brackets in '{non_html_tag}' → '&lt;{non_html_tag[1:-1]}&gt;'")
        df = non_html_processor.encode_brackets(
            df, non_html_tag, '&lt;', '&gt;', scope='global'
        )
    
    print(f"✓ Encoded {len(non_html_results['Non_HTML_Tags_Summary'])} non-HTML tags")
    
else:
    print("No non-HTML tags found!")

# Save to Excel and SQLite
save_to_excel(non_html_results, "05_non_html_tag_analysis.xlsx")
save_intermediate_stage_sqlite(df, "05_non_html_encoding", conn)

print("Step 5 complete: Non-HTML tag detection and encoding")

Step 5: Analyzing non-HTML tags...
Scanning for non-HTML content in angle brackets...
Found 49 non-HTML tags for encoding
Non-HTML Tags Summary:


,Non_HTML_Tag,Count,Locations,Sample_Context
3,<Concanauhtli>,3,"Row 2158, Col Attestations from sources in Eng...",nauhtli (Zoquicanauhtli) is the same as the go...
9,"<TLACOOCELUTL: [...] yoan qujtocaiotia, tlacom...",3,"Row 5139, Col Attestations from sources in Eng...","tl*, as the following passage (FC 11: 3) impli..."
16,<ue>,3,"Row 10787, Col Alonso de Molina; Row 16924, Co...",ret. onitemiquiztlapopolhui.) perdonar la muer...
18,<Canauhtli>,2,"Row 26094, Col Attestations from sources in En...",dum in Paragraph Three [FC: 57] elaborates: “D...
22,<Ā-COYO-TL>,2,"Row 31289, Col Attestations from sources in En...","omeone beat the two-toned drum.” However, acoy..."
2,<n>,2,"Row 1408, Col Alonso de Molina; Row 12954, Col...",<p>aami. n. (pret. onaan.) mo[[[<n>]]]tear o c...
23,<Ā-TŌTO-LIN>,2,"Row 31289, Col Attestations from sources in En...",Ā-COYO-TL> is the Neotropic Cormorant and atot...
29,<ZŌL-IN>,2,"Row 31335, Col Attestations from sources in En...",49 Ooaton] “It is the same as the Montezuma Qu...
0,<sip>,1,"Row 1137, Col Attestations from sources in Eng...","07), 157.</bibl> Escrivano de cabil tlsn— fabi..."
1,<susp>,1,"Row 1137, Col Attestations from sources in Eng...","erican Center, 1976), Doc. 8.</bibl> diego jua..."



Applying bracket encoding...
Encoding brackets in '<Concanauhtli>' → '&lt;Concanauhtli&gt;'
  Column 'Attestations from sources in English': 3 encodings
Total encodings made: 3
Encoding brackets in '<TLACOOCELUTL: [...] yoan qujtocaiotia, tlacomjztli, tepiton, pachtontli, melacpil: vel ixqujch, in castillan mjzton, nexeoac, ticeoac, cujcujltic, ocelocujcujltic, molchachapatz>' → '&lt;TLACOOCELUTL: [...] yoan qujtocaiotia, tlacomjztli, tepiton, pachtontli, melacpil: vel ixqujch, in castillan mjzton, nexeoac, ticeoac, cujcujltic, ocelocujcujltic, molchachapatz&gt;'
  Column 'Attestations from sources in English': 3 encodings
Total encodings made: 3
Encoding brackets in '<ue>' → '&lt;ue&gt;'
  Column 'Alonso de Molina': 3 encodings
Total encodings made: 3
Encoding brackets in '<Canauhtli>' → '&lt;Canauhtli&gt;'
  Column 'Attestations from sources in English': 2 encodings
Total encodings made: 2
Encoding brackets in '<Ā-COYO-TL>' → '&lt;Ā-COYO-TL&gt;'
  Column 'Attestations from sources i

## Step 6: Citation Extraction

In [28]:
class CitationExtractor:
    def __init__(self):
        pass

    def extract_bibl_content(self, text: str) -> List[str]:
        """Extract from clean bibl tags (post Steps 1-5)"""
        if pd.isna(text) or text == '':
            return []

        text_str = str(text)
        
        # Simple pattern for well-formed bibl tags
        pattern = r'<bibl[^>]*>(.*?)</bibl>'
        matches = re.findall(pattern, text_str, re.DOTALL | re.IGNORECASE)
        
        cleaned_citations = []
        for match in matches:
            # Clean up the content
            cleaned = re.sub(r'<[^>]+>', '', match)  # Remove nested HTML
            cleaned = ' '.join(cleaned.strip().split())  # Normalize whitespace
            
            if cleaned and len(cleaned.strip()) > 5:  # Must be substantial
                cleaned_citations.append(cleaned.strip())
        
        return cleaned_citations

    def add_citation_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add citation columns to cleaned dataframe"""
        print("Adding citation columns...")
        df_with_citations = df.copy()

        # Initialize new columns
        df_with_citations['Citations'] = ''
        df_with_citations['Number_of_Citations'] = 0

        for idx, row in df_with_citations.iterrows():
            if idx % 5000 == 0:
                print(f"  Processing row {idx}...")
                
            all_citations = []

            # Extract from all original columns
            for col in df.columns:  # Original columns only
                cell_value = row[col]
                if pd.notna(cell_value):
                    citations = self.extract_bibl_content(cell_value)
                    all_citations.extend(citations)

            # Set new column values
            if all_citations:
                df_with_citations.loc[idx, 'Citations'] = ' | '.join(all_citations)
                df_with_citations.loc[idx, 'Number_of_Citations'] = len(all_citations)

        return df_with_citations

def process_citations_step6(conn: sqlite3.Connection) -> pd.DataFrame:
    """Step 6: Citation extraction using cleaned data from Step 5"""
    print("Step 6: Citation Extraction")
    
    # Load the CLEANED data from Step 5
    print("  - Loading cleaned data from Step 5...")
    df_clean = pd.read_sql("SELECT * FROM [05_non_html_encoding_stage]", conn)
    print(f"  - Loaded {len(df_clean)} rows")
    
    # Extract citations from clean data
    citation_extractor = CitationExtractor()
    df_with_citations = citation_extractor.add_citation_columns(df_clean)
    
    # Save processed stage
    save_intermediate_stage_sqlite(df_with_citations, "06_citation_extraction_stage", conn)
    
    # Quick stats
    rows_with_citations = len(df_with_citations[df_with_citations['Number_of_Citations'] > 0])
    total_citations = df_with_citations['Number_of_Citations'].sum()
    
    print(f"  - Rows with citations: {rows_with_citations}")
    print(f"  - Total citations extracted: {total_citations}")
    
    return df_with_citations

In [29]:
df_with_citations = process_citations_step6(conn)

Step 6: Citation Extraction
  - Loading cleaned data from Step 5...
  - Loaded 31806 rows
Adding citation columns...
  Processing row 0...
  Processing row 5000...
  Processing row 10000...
  Processing row 15000...
  Processing row 20000...
  Processing row 25000...
  Processing row 30000...
Saved to SQLite table: 06_citation_extraction_stage_stage
  - Rows with citations: 31281
  - Total citations extracted: 53497


## Step 7: Cross-Reference Extraction

In [30]:
class CrossReferenceExtractor:
    def __init__(self):
        # Common cross-reference patterns in academic/dictionary entries
        self.crossref_patterns = [
            r'\(see\s+([^)]+)\)',           # (see something)
            r'\(cf\.\s+([^)]+)\)',          # (cf. something)  
            r'\(compare\s+([^)]+)\)',       # (compare something)
            r'\(also\s+([^)]+)\)',          # (also something)
            r'\(variant\s+of\s+([^)]+)\)',  # (variant of something)
            r'\(see\s+also\s+([^)]+)\)'     # (see also something)
        ]

    def extract_cross_references(self, text: str) -> List[str]:
        """Extract cross-references from text"""
        if pd.isna(text) or text == '':
            return []

        text_str = str(text)
        crossrefs = []

        # Extract using all patterns
        for pattern in self.crossref_patterns:
            matches = re.findall(pattern, text_str, re.IGNORECASE)
            for match in matches:
                cleaned = self._clean_crossref_content(match)
                if cleaned and cleaned not in crossrefs:  # Avoid duplicates
                    crossrefs.append(cleaned)

        return crossrefs

    def _clean_crossref_content(self, content: str) -> str:
        """Clean up extracted cross-reference content"""
        if not content:
            return ""
        
        # Remove any HTML tags
        cleaned = re.sub(r'<[^>]+>', '', content)
        
        # Clean up whitespace
        cleaned = ' '.join(cleaned.strip().split())
        
        # Remove trailing punctuation
        cleaned = cleaned.strip('.,;:')
        
        # Must be substantial and start with a letter/number
        if len(cleaned) > 1 and cleaned[0].isalnum():
            return cleaned.strip()
        
        return ""

    def _classify_crossref(self, crossref: str) -> str:
        """Classify the type of cross-reference"""
        crossref_lower = crossref.lower()
        
        if any(word in crossref_lower for word in ['molina', 'karttunen', 'carochi']):
            return 'Scholar_Reference'
        elif len(crossref.split()) == 1:
            return 'Single_Word'
        elif len(crossref.split()) <= 3:
            return 'Short_Phrase'
        else:
            return 'Long_Phrase'

    def analyze_crossrefs_in_dataframe(self, df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
        """Analyze all cross-references in the dataframe"""
        crossref_details = []
        row_summaries = []
        
        print("Analyzing cross-references across dataframe...")
        
        for idx, row in df.iterrows():
            if idx % 5000 == 0:
                print(f"  Processing row {idx}...")
            
            row_crossrefs = []
            
            # Check each column for cross-references
            for col in df.columns:
                # Skip the columns we just added in Step 6
                if col in ['Citations', 'Number_of_Citations']:
                    continue
                    
                cell_value = row[col]
                if pd.notna(cell_value) and str(cell_value).strip():
                    crossrefs = self.extract_cross_references(cell_value)
                    
                    for crossref in crossrefs:
                        crossref_details.append({
                            'Row_Index': idx,
                            'Row_ID': row.get('Ref', f'Row_{idx}'),
                            'Column': col,
                            'Cross_Reference': crossref,
                            'Crossref_Length': len(crossref),
                            'Crossref_Type': self._classify_crossref(crossref)
                        })
                        row_crossrefs.append(crossref)
            
            # Summary for this row
            if row_crossrefs:
                row_summaries.append({
                    'Row_Index': idx,
                    'Row_ID': row.get('Ref', f'Row_{idx}'),
                    'Headword': row.get('Headword', ''),
                    'Total_Cross_References': len(row_crossrefs),
                    'Cross_References_Preview': ' | '.join(row_crossrefs[:3]) + ('...' if len(row_crossrefs) > 3 else '')
                })
        
        # Overall statistics
        total_crossrefs = len(crossref_details)
        rows_with_crossrefs = len(row_summaries)
        
        # Count by type
        type_counts = {}
        for detail in crossref_details:
            crossref_type = detail['Crossref_Type']
            type_counts[crossref_type] = type_counts.get(crossref_type, 0) + 1
        
        overall_stats = pd.DataFrame([{
            'Total_Rows_Processed': len(df),
            'Rows_With_Cross_References': rows_with_crossrefs,
            'Total_Cross_References_Found': total_crossrefs,
            'Single_Word_Refs': type_counts.get('Single_Word', 0),
            'Short_Phrase_Refs': type_counts.get('Short_Phrase', 0),
            'Scholar_Refs': type_counts.get('Scholar_Reference', 0),
            'Avg_Crossrefs_Per_Row': total_crossrefs / rows_with_crossrefs if rows_with_crossrefs > 0 else 0,
            'Max_Crossrefs_Per_Row': max([r['Total_Cross_References'] for r in row_summaries]) if row_summaries else 0
        }])
        
        return {
            'CrossRef_Details': pd.DataFrame(crossref_details),
            'Row_Summaries': pd.DataFrame(row_summaries),
            'Overall_Stats': overall_stats
        }

    def add_crossref_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add cross-reference columns WITHOUT removing original content"""
        print("Adding cross-reference columns...")
        df_with_crossrefs = df.copy()

        # Initialize new columns
        df_with_crossrefs['Cross_References'] = ''
        df_with_crossrefs['Number_of_Cross_References'] = 0
        df_with_crossrefs['CrossRef_Types'] = ''

        for idx, row in df_with_crossrefs.iterrows():
            if idx % 5000 == 0:
                print(f"  Processing row {idx}...")
                
            all_crossrefs = []
            crossref_types = []

            # Extract from original columns (skip our new citation columns)
            for col in df.columns:
                if col in ['Citations', 'Number_of_Citations']:
                    continue
                    
                cell_value = row[col]
                if pd.notna(cell_value) and str(cell_value).strip():
                    crossrefs = self.extract_cross_references(cell_value)
                    all_crossrefs.extend(crossrefs)
                    
                    # Track cross-reference types
                    for crossref in crossrefs:
                        crossref_types.append(self._classify_crossref(crossref))

            # Set new column values
            if all_crossrefs:
                df_with_crossrefs.loc[idx, 'Cross_References'] = ' | '.join(all_crossrefs)
                df_with_crossrefs.loc[idx, 'Number_of_Cross_References'] = len(all_crossrefs)
                df_with_crossrefs.loc[idx, 'CrossRef_Types'] = ', '.join(set(crossref_types))

        return df_with_crossrefs

def process_crossrefs_step7(conn: sqlite3.Connection) -> pd.DataFrame:
    """Step 7: Cross-reference extraction using data from Step 6"""
    print("Step 7: Cross-Reference Extraction")
    
    # Load data from Step 6 (includes citations)
    print("  - Loading data from Step 6...")
    df_with_citations = pd.read_sql("SELECT * FROM [06_citation_extraction_stage_stage]", conn)
    print(f"  - Loaded {len(df_with_citations)} rows")
    
    # Extract cross-references
    crossref_extractor = CrossReferenceExtractor()
    
    # Optional: Analyze first for insights
    print("  - Analyzing cross-references...")
    crossref_analysis = crossref_extractor.analyze_crossrefs_in_dataframe(df_with_citations)
    
    # Save analysis to SQLite
    crossref_analysis['CrossRef_Details'].to_sql('07_crossref_details', conn, if_exists='replace', index=False)
    crossref_analysis['Row_Summaries'].to_sql('07_crossref_summary', conn, if_exists='replace', index=False)
    crossref_analysis['Overall_Stats'].to_sql('07_crossref_stats', conn, if_exists='replace', index=False)
    
    # Add cross-reference columns
    df_with_crossrefs = crossref_extractor.add_crossref_columns(df_with_citations)
    
    # Save processed stage
    save_intermediate_stage_sqlite(df_with_crossrefs, "07_crossref_extraction_stage", conn)
    
    # Print results
    stats = crossref_analysis['Overall_Stats'].iloc[0]
    print(f"  - Rows with cross-references: {stats['Rows_With_Cross_References']}")
    print(f"  - Total cross-references found: {stats['Total_Cross_References_Found']}")
    print(f"  - Single word refs: {stats['Single_Word_Refs']}")
    print(f"  - Short phrase refs: {stats['Short_Phrase_Refs']}")
    print(f"  - Scholar refs: {stats['Scholar_Refs']}")
    print(f"  - Max cross-refs per row: {stats['Max_Crossrefs_Per_Row']}")
    
    return df_with_crossrefs

In [31]:
# Step 7: Cross-Reference Extraction
df_with_crossrefs = process_crossrefs_step7(conn)

Step 7: Cross-Reference Extraction
  - Loading data from Step 6...
  - Loaded 31806 rows
  - Analyzing cross-references...
Analyzing cross-references across dataframe...
  Processing row 0...
  Processing row 5000...
  Processing row 10000...
  Processing row 15000...
  Processing row 20000...
  Processing row 25000...
  Processing row 30000...
Adding cross-reference columns...
  Processing row 0...
  Processing row 5000...
  Processing row 10000...
  Processing row 15000...
  Processing row 20000...
  Processing row 25000...
  Processing row 30000...
Saved to SQLite table: 07_crossref_extraction_stage_stage
  - Rows with cross-references: 16991.0
  - Total cross-references found: 17665.0
  - Single word refs: 1231.0
  - Short phrase refs: 371.0
  - Scholar refs: 15886.0
  - Max cross-refs per row: 4.0


In [34]:
# Check what types of cross-references we're finding
crossref_details = pd.read_sql("SELECT * FROM [07_crossref_details] LIMIT 10", conn)
print("Sample cross-references found:")
print(crossref_details[['Cross_Reference', 'Crossref_Type']].head(10))

# Check the scholar references specifically
scholar_refs = pd.read_sql("""
    SELECT Cross_Reference, COUNT(*) as count 
    FROM [07_crossref_details] 
    WHERE Crossref_Type = 'Scholar_Reference' 
    GROUP BY Cross_Reference 
    ORDER BY count DESC 
    LIMIT 10
""", conn)
print("\nTop scholar references:")
print(scholar_refs)

# Check some examples of different patterns
print("\nSample by pattern type:")
for pattern_type in ['Single_Word', 'Short_Phrase', 'Scholar_Reference']:
    sample = pd.read_sql(f"""
        SELECT Cross_Reference 
        FROM [07_crossref_details] 
        WHERE Crossref_Type = '{pattern_type}' 
        LIMIT 3
    """, conn)
    print(f"{pattern_type}: {list(sample['Cross_Reference'])}")

Sample cross-references found:
                          Cross_Reference      Crossref_Type
0                                  Molina  Scholar_Reference
1                                Lockhart        Single_Word
2        Molina, Karttunen, Lockhart, etc  Scholar_Reference
3                               Karttunen  Scholar_Reference
4  Molina; and see our entry for achitzin  Scholar_Reference
5                 also our entry for achi        Long_Phrase
6                      our entry for achi        Long_Phrase
7                                  Molina  Scholar_Reference
8                                  Molina  Scholar_Reference
9                            Molina et al  Scholar_Reference

Top scholar references:
                   Cross_Reference  count
0                           Molina  10816
1                        Karttunen   4062
2             Molina and Karttunen    336
3             Karttunen and Molina    136
4          Molina and attestations     50
5                    

## Step 8: Complete Workflow Example

In [38]:
def complete_processing_workflow_sqlite(conn: sqlite3.Connection, 
                                     validate_steps: bool = True,
                                     create_final_reports: bool = True) -> pd.DataFrame:
    """
    Step 8: Complete Workflow Integration for SQLite-based processing
    
    This function validates all previous steps, creates comprehensive reports,
    and produces the final processed dataset.
    """
    
    print("=" * 70)
    print("STEP 8: COMPLETE WORKFLOW INTEGRATION")
    print("=" * 70)
    
    # Step 8.1: Validate Pipeline Integrity
    if validate_steps:
        print("\n8.1 Validating pipeline integrity...")
        validation_results = validate_processing_pipeline(conn)
        
        if not validation_results['pipeline_valid']:
            print("❌ Pipeline validation failed!")
            for error in validation_results['errors']:
                print(f"   - {error}")
            return None
        else:
            print("✅ Pipeline validation passed!")
            print(f"   - All {validation_results['stages_validated']} stages verified")
    
    # Step 8.2: Load Final Processed Data
    print("\n8.2 Loading final processed data...")
    try:
        # Load from Step 7 (cross-reference extraction)
        final_df = pd.read_sql("SELECT * FROM [07_crossref_extraction_stage_stage]", conn)
        print(f"   - Loaded {len(final_df)} rows with {len(final_df.columns)} columns")
        
        # Quick data integrity check
        original_count = pd.read_sql("SELECT COUNT(*) as count FROM WHP_EarlyNahuatl_Data", conn).iloc[0]['count']
        if len(final_df) != original_count:
            print(f"⚠️  Row count mismatch: Original {original_count}, Final {len(final_df)}")
        
    except Exception as e:
        print(f"❌ Error loading final data: {e}")
        return None
    
    # Step 8.3: Create Comprehensive Analysis Reports
    if create_final_reports:
        print("\n8.3 Creating comprehensive analysis reports...")
        analysis_reports = create_comprehensive_reports(conn, final_df)
        
        # Save all reports to SQLite
        for report_name, report_df in analysis_reports.items():
            table_name = f"08_final_report_{report_name}"
            report_df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"   - Saved {report_name} report to {table_name}")
    
    # Step 8.4: Create Final Output Files
    print("\n8.4 Creating final output files...")
    output_files = create_final_output_files(final_df, analysis_reports if create_final_reports else {})
    
    # Step 8.5: Save Final Stage to SQLite
    print("\n8.5 Saving final processed dataset...")
    save_intermediate_stage_sqlite(final_df, "08_final_complete_dataset", conn)
    
    # Step 8.6: Print Final Summary
    print_final_processing_summary(conn, final_df, validation_results if validate_steps else None)
    
    return final_df

def validate_processing_pipeline(conn: sqlite3.Connection) -> Dict:
    """Validate that all processing steps completed successfully"""
    
    required_stages = [
        "WHP_EarlyNahuatl_Data",  # Original data
        "04_malformed_repair_stage",  # Step 4
        "05_non_html_encoding_stage",  # Step 5  
        "06_citation_extraction_stage_stage",  # Step 6
        "07_crossref_extraction_stage_stage"  # Step 7
    ]
    
    validation_results = {
        'pipeline_valid': True,
        'errors': [],
        'stages_validated': 0,
        'stage_details': {}
    }
    
    # Check if all required tables exist
    tables_query = "SELECT name FROM sqlite_master WHERE type='table'"
    existing_tables = pd.read_sql(tables_query, conn)['name'].tolist()
    
    for stage in required_stages:
        if stage not in existing_tables:
            validation_results['pipeline_valid'] = False
            validation_results['errors'].append(f"Missing table: {stage}")
        else:
            # Check row count consistency
            try:
                count = pd.read_sql(f"SELECT COUNT(*) as count FROM [{stage}]", conn).iloc[0]['count']
                validation_results['stage_details'][stage] = {'row_count': count}
                validation_results['stages_validated'] += 1
            except Exception as e:
                validation_results['pipeline_valid'] = False
                validation_results['errors'].append(f"Error accessing {stage}: {e}")
    
    # Validate row count consistency across stages
    if validation_results['pipeline_valid']:
        original_count = validation_results['stage_details']['WHP_EarlyNahuatl_Data']['row_count']
        for stage, details in validation_results['stage_details'].items():
            if stage != 'WHP_EarlyNahuatl_Data' and details['row_count'] != original_count:
                validation_results['errors'].append(
                    f"Row count mismatch in {stage}: {details['row_count']} vs {original_count}"
                )
    
    return validation_results

def create_comprehensive_reports(conn: sqlite3.Connection, final_df: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    """Create comprehensive analysis reports"""
    
    reports = {}
    
    # 1. Processing Summary Report
    original_df = pd.read_sql("SELECT * FROM WHP_EarlyNahuatl_Data LIMIT 1", conn)
    
    reports['processing_summary'] = pd.DataFrame([{
        'Original_Rows': pd.read_sql("SELECT COUNT(*) as count FROM WHP_EarlyNahuatl_Data", conn).iloc[0]['count'],
        'Original_Columns': len(original_df.columns),
        'Final_Rows': len(final_df),
        'Final_Columns': len(final_df.columns),
        'Columns_Added': len(final_df.columns) - len(original_df.columns),
        'Processing_Date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    }])
    
    # 2. Citation Analysis Report
    citation_stats = {
        'Total_Rows_With_Citations': len(final_df[final_df['Number_of_Citations'] > 0]),
        'Total_Citations_Extracted': final_df['Number_of_Citations'].sum(),
        'Max_Citations_Per_Row': final_df['Number_of_Citations'].max(),
        'Avg_Citations_Per_Row': final_df['Number_of_Citations'].mean(),
        'Median_Citations_Per_Row': final_df['Number_of_Citations'].median()
    }
    reports['citation_analysis'] = pd.DataFrame([citation_stats])
    
    # 3. Cross-Reference Analysis Report  
    crossref_stats = {
        'Total_Rows_With_CrossRefs': len(final_df[final_df['Number_of_Cross_References'] > 0]),
        'Total_CrossRefs_Extracted': final_df['Number_of_Cross_References'].sum(),
        'Max_CrossRefs_Per_Row': final_df['Number_of_Cross_References'].max(),
        'Avg_CrossRefs_Per_Row': final_df['Number_of_Cross_References'].mean(),
        'Median_CrossRefs_Per_Row': final_df['Number_of_Cross_References'].median()
    }
    reports['crossref_analysis'] = pd.DataFrame([crossref_stats])
    
    # 4. Data Quality Report
    quality_metrics = []
    for col in final_df.columns:
        quality_metrics.append({
            'Column_Name': col,
            'Data_Type': str(final_df[col].dtype),
            'Non_Null_Count': final_df[col].count(),
            'Null_Count': final_df[col].isnull().sum(),
            'Null_Percentage': (final_df[col].isnull().sum() / len(final_df)) * 100,
            'Unique_Values': final_df[col].nunique() if final_df[col].dtype == 'object' else 'N/A'
        })
    reports['data_quality'] = pd.DataFrame(quality_metrics)
    
    # 5. Column Mapping Report (original vs final)
    original_columns = pd.read_sql("PRAGMA table_info(WHP_EarlyNahuatl_Data)", conn)['name'].tolist()
    final_columns = final_df.columns.tolist()
    
    column_mapping = []
    for col in original_columns:
        column_mapping.append({
            'Column_Name': col,
            'Status': 'Original',
            'Present_In_Final': col in final_columns
        })
    
    for col in final_columns:
        if col not in original_columns:
            column_mapping.append({
                'Column_Name': col,
                'Status': 'Added_During_Processing',
                'Present_In_Final': True
            })
    
    reports['column_mapping'] = pd.DataFrame(column_mapping)
    
    return reports

def create_final_output_files(final_df: pd.DataFrame, analysis_reports: Dict) -> Dict[str, str]:
    """Create final output files"""
    
    output_files = {}
    
    # 1. Final processed dataset (CSV)
    csv_filename = f"final_nahuatl_dataset_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
    csv_path = os.path.join('working_files', csv_filename)
    final_df.to_csv(csv_path, index=False)
    output_files['dataset_csv'] = csv_path
    print(f"   - Saved final dataset: {csv_path}")
    
    # 2. Analysis reports (Excel)
    if analysis_reports:
        excel_filename = f"comprehensive_analysis_report_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        excel_path = os.path.join('working_files', excel_filename)
        save_to_excel(analysis_reports, excel_filename)
        output_files['analysis_excel'] = excel_path
        print(f"   - Saved analysis reports: {excel_path}")
    
    # 3. Citation-only dataset
    citations_df = final_df[final_df['Number_of_Citations'] > 0][
        ['Ref', 'Headword', 'Citations', 'Number_of_Citations']
    ].copy()
    citations_filename = f"citations_extract_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
    citations_path = os.path.join('working_files', citations_filename)
    citations_df.to_csv(citations_path, index=False)
    output_files['citations_csv'] = citations_path
    print(f"   - Saved citations extract: {citations_path}")
    
    # 4. Cross-references-only dataset
    crossrefs_df = final_df[final_df['Number_of_Cross_References'] > 0][
        ['Ref', 'Headword', 'Cross_References', 'Number_of_Cross_References']
    ].copy()
    crossrefs_filename = f"crossrefs_extract_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
    crossrefs_path = os.path.join('working_files', crossrefs_filename)
    crossrefs_df.to_csv(crossrefs_path, index=False)
    output_files['crossrefs_csv'] = crossrefs_path
    print(f"   - Saved cross-references extract: {crossrefs_path}")
    
    return output_files

def print_final_processing_summary(conn: sqlite3.Connection, final_df: pd.DataFrame, validation_results: Dict = None):
    """Print comprehensive processing summary"""
    
    print("\n" + "=" * 70)
    print("PROCESSING COMPLETE - FINAL SUMMARY")
    print("=" * 70)
    
    # Original vs Final comparison
    original_count = pd.read_sql("SELECT COUNT(*) as count FROM WHP_EarlyNahuatl_Data", conn).iloc[0]['count']
    original_cols = len(pd.read_sql("PRAGMA table_info(WHP_EarlyNahuatl_Data)", conn))
    
    print(f"📊 DATA TRANSFORMATION:")
    print(f"   Original: {original_count:,} rows × {original_cols} columns")
    print(f"   Final:    {len(final_df):,} rows × {len(final_df.columns)} columns")
    print(f"   Added:    {len(final_df.columns) - original_cols} new columns")
    
    # Citation statistics
    citation_rows = len(final_df[final_df['Number_of_Citations'] > 0])
    total_citations = final_df['Number_of_Citations'].sum()
    print(f"\n📚 CITATIONS:")
    print(f"   Rows with citations: {citation_rows:,} ({citation_rows/len(final_df)*100:.1f}%)")
    print(f"   Total citations:     {total_citations:,}")
    print(f"   Max per row:         {final_df['Number_of_Citations'].max()}")
    
    # Cross-reference statistics
    crossref_rows = len(final_df[final_df['Number_of_Cross_References'] > 0])
    total_crossrefs = final_df['Number_of_Cross_References'].sum()
    print(f"\n🔗 CROSS-REFERENCES:")
    print(f"   Rows with cross-refs: {crossref_rows:,} ({crossref_rows/len(final_df)*100:.1f}%)")
    print(f"   Total cross-refs:     {total_crossrefs:,}")
    print(f"   Max per row:          {final_df['Number_of_Cross_References'].max()}")
    
    # Validation summary
    if validation_results:
        print(f"\n✅ VALIDATION:")
        print(f"   Pipeline stages validated: {validation_results['stages_validated']}")
        print(f"   Data integrity:            {'✅ PASSED' if validation_results['pipeline_valid'] else '❌ FAILED'}")
    
    # Database tables summary
    tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)
    print(f"\n💾 SQLITE DATABASE:")
    print(f"   Total tables created: {len(tables)}")
    print("   Final dataset table:  08_final_complete_dataset_stage")
    
    print("\n" + "=" * 70)
    print("🎉 NAHUATL DICTIONARY PROCESSING PIPELINE COMPLETE!")
    print("=" * 70)

In [39]:
# Step 8: Complete Workflow Integration
final_dataset = complete_processing_workflow_sqlite(
    conn=conn,
    validate_steps=True,
    create_final_reports=True
)

# Optional: Run quick validation check
print("\nQuick validation check:")
print(f"Final dataset shape: {final_dataset.shape}")
print(f"Citations column exists: {'Citations' in final_dataset.columns}")
print(f"Cross-references column exists: {'Cross_References' in final_dataset.columns}")

STEP 8: COMPLETE WORKFLOW INTEGRATION

8.1 Validating pipeline integrity...
✅ Pipeline validation passed!
   - All 5 stages verified

8.2 Loading final processed data...
   - Loaded 31806 rows with 18 columns

8.3 Creating comprehensive analysis reports...
   - Saved processing_summary report to 08_final_report_processing_summary
   - Saved citation_analysis report to 08_final_report_citation_analysis
   - Saved crossref_analysis report to 08_final_report_crossref_analysis
   - Saved data_quality report to 08_final_report_data_quality
   - Saved column_mapping report to 08_final_report_column_mapping

8.4 Creating final output files...
   - Saved final dataset: working_files\final_nahuatl_dataset_20250910_075418.csv
Saved to: working_files\comprehensive_analysis_report_20250910_075420.xlsx
   - Saved analysis reports: working_files\comprehensive_analysis_report_20250910_075420.xlsx
   - Saved citations extract: working_files\citations_extract_20250910_075420.csv
   - Saved cross-refere

## Individual Processing Functions

In [44]:
# ==========================================
# STEP 9: MANUAL PROCESSING UTILITIES
# ==========================================

class ManualProcessingToolkit:
    """Manual processing tools for edge cases and fine-tuning"""
    
    def __init__(self, conn: sqlite3.Connection):
        self.conn = conn
        
    def repair_specific_tag(self, tag_to_repair: str, replacement: str, 
                           scope: str = 'global', 
                           column: str = None, 
                           row: int = None,
                           table_name: str = "08_final_complete_dataset_stage") -> pd.DataFrame:
        """
        Repair specific malformed tags with granular control
        
        Scopes:
        - 'global': Throughout entire dataset
        - 'column': Only in specified column  
        - 'row': Only in specified row
        - 'cell': Only in specific cell (column + row)
        """
        
        print(f"🔧 MANUAL TAG REPAIR")
        print(f"   Target: '{tag_to_repair}' → '{replacement}'")
        print(f"   Scope: {scope}")
        
        # Load current dataset
        df = pd.read_sql(f"SELECT * FROM [{table_name}]", self.conn)
        df_repaired = df.copy()
        
        changes_made = 0
        
        if scope == 'global':
            # Replace throughout entire DataFrame
            for col in df_repaired.columns:
                original_values = df_repaired[col].astype(str)
                df_repaired[col] = original_values.str.replace(
                    tag_to_repair, replacement, regex=False
                )
                # Count changes
                changes_made += (original_values != df_repaired[col].astype(str)).sum()
                
        elif scope == 'column' and column:
            if column in df_repaired.columns:
                original_values = df_repaired[column].astype(str)
                df_repaired[column] = original_values.str.replace(
                    tag_to_repair, replacement, regex=False
                )
                changes_made = (original_values != df_repaired[column].astype(str)).sum()
            else:
                print(f"   ❌ Column '{column}' not found")
                return df
                
        elif scope == 'row' and row is not None:
            for col in df_repaired.columns:
                if pd.notna(df_repaired.loc[row, col]):
                    original_value = str(df_repaired.loc[row, col])
                    new_value = original_value.replace(tag_to_repair, replacement)
                    if original_value != new_value:
                        df_repaired.loc[row, col] = new_value
                        changes_made += 1
                        
        elif scope == 'cell' and column and row is not None:
            if column in df_repaired.columns and row < len(df_repaired):
                if pd.notna(df_repaired.loc[row, column]):
                    original_value = str(df_repaired.loc[row, column])
                    new_value = original_value.replace(tag_to_repair, replacement)
                    if original_value != new_value:
                        df_repaired.loc[row, column] = new_value
                        changes_made = 1
        
        print(f"   ✅ Made {changes_made} changes")
        
        # Save repaired dataset
        if changes_made > 0:
            timestamp = pd.Timestamp.now().strftime('%H%M%S')
            repair_table = f"09_manual_repair_{scope}_{timestamp}_stage"
            save_intermediate_stage_sqlite(df_repaired, repair_table, self.conn)
            print(f"   💾 Saved to: {repair_table}")
        
        return df_repaired
    
    def batch_repair_tags(self, repair_dict: Dict[str, str], 
                         scope: str = 'global',
                         table_name: str = "08_final_complete_dataset_stage") -> pd.DataFrame:
        """Apply multiple tag repairs in batch"""
        
        print(f"🔧 BATCH TAG REPAIR ({len(repair_dict)} repairs)")
        
        df = pd.read_sql(f"SELECT * FROM [{table_name}]", self.conn)
        df_repaired = df.copy()
        
        total_changes = 0
        
        for tag_to_repair, replacement in repair_dict.items():
            print(f"   Repairing: '{tag_to_repair}' → '{replacement}'")
            
            if scope == 'global':
                for col in df_repaired.columns:
                    original_values = df_repaired[col].astype(str)
                    df_repaired[col] = original_values.str.replace(
                        tag_to_repair, replacement, regex=False
                    )
                    changes = (original_values != df_repaired[col].astype(str)).sum()
                    total_changes += changes
                    if changes > 0:
                        print(f"     ✅ {changes} changes in {col}")
        
        print(f"   🎯 Total changes: {total_changes}")
        
        # Save batch repaired dataset
        if total_changes > 0:
            timestamp = pd.Timestamp.now().strftime('%H%M%S')
            batch_table = f"09_batch_repair_{timestamp}_stage"
            save_intermediate_stage_sqlite(df_repaired, batch_table, self.conn)
            print(f"   💾 Saved to: {batch_table}")
        
        return df_repaired
    
    def reanalyze_after_changes(self, table_name: str = None) -> Dict:
        """Re-run analysis after manual changes"""
        
        print("🔍 RE-ANALYZING AFTER MANUAL CHANGES")
        
        if not table_name:
            # Find the most recent manual repair table
            tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' AND name LIKE '09_%' ORDER BY name DESC", self.conn)
            if not tables.empty:
                table_name = tables.iloc[0]['name']
                print(f"   Using latest manual repair table: {table_name}")
            else:
                table_name = "08_final_complete_dataset_stage"
                print(f"   Using final dataset table: {table_name}")
        
        df = pd.read_sql(f"SELECT * FROM [{table_name}]", self.conn)
        
        # Re-run analysis (simplified version for quick check)
        analysis_results = {
            'total_rows': len(df),
            'total_columns': len(df.columns),
            'citation_stats': {
                'rows_with_citations': len(df[df['Number_of_Citations'] > 0]) if 'Number_of_Citations' in df.columns else 0,
                'total_citations': df['Number_of_Citations'].sum() if 'Number_of_Citations' in df.columns else 0
            },
            'crossref_stats': {
                'rows_with_crossrefs': len(df[df['Number_of_Cross_References'] > 0]) if 'Number_of_Cross_References' in df.columns else 0,
                'total_crossrefs': df['Number_of_Cross_References'].sum() if 'Number_of_Cross_References' in df.columns else 0
            }
        }
        
        print(f"   📊 Rows: {analysis_results['total_rows']:,}")
        print(f"   📚 Citations: {analysis_results['citation_stats']['total_citations']:,}")
        print(f"   🔗 Cross-refs: {analysis_results['crossref_stats']['total_crossrefs']:,}")
        
        return analysis_results
    
    def find_problem_entries(self, search_pattern: str, 
                           table_name: str = "08_final_complete_dataset_stage") -> pd.DataFrame:
        """Find entries containing specific patterns for manual review"""
        
        print(f"🔍 FINDING PROBLEM ENTRIES: '{search_pattern}'")
        
        df = pd.read_sql(f"SELECT * FROM [{table_name}]", self.conn)
        
        problem_entries = []
        
        for idx, row in df.iterrows():
            for col in df.columns:
                cell_value = str(row[col])
                if search_pattern in cell_value:
                    problem_entries.append({
                        'Row_Index': idx,
                        'Row_ID': row.get('Ref', f'Row_{idx}'),
                        'Column': col,
                        'Headword': row.get('Headword', ''),
                        'Problem_Context': cell_value[:200] + '...' if len(cell_value) > 200 else cell_value
                    })
        
        problem_df = pd.DataFrame(problem_entries)
        
        if not problem_df.empty:
            print(f"   🎯 Found {len(problem_df)} problem instances")
            
            # Save problem entries for review
            timestamp = pd.Timestamp.now().strftime('%H%M%S')
            problem_table = f"09_problem_entries_{timestamp}"
            problem_df.to_sql(problem_table, self.conn, if_exists='replace', index=False)
            print(f"   💾 Saved problem entries to: {problem_table}")
        else:
            print(f"   ✅ No instances of '{search_pattern}' found")
        
        return problem_df

def create_manual_processing_toolkit(conn: sqlite3.Connection) -> ManualProcessingToolkit:
    """Initialize manual processing toolkit"""
    return ManualProcessingToolkit(conn)

# ==========================================
# STEP 9 USAGE EXAMPLES
# ==========================================

def step9_usage_examples(conn: sqlite3.Connection):
    """Examples of manual processing tools"""
    
    print("=" * 70)
    print("STEP 9: MANUAL PROCESSING TOOLKIT EXAMPLES")
    print("=" * 70)
    
    toolkit = create_manual_processing_toolkit(conn)
    
    # Example 1: Fix a specific malformed tag
    print("\n9.1 Example: Fix specific malformed tag")
    # toolkit.repair_specific_tag("</bibbl>", "</bibl>", scope='global')
    
    # Example 2: Batch repair multiple issues
    print("\n9.2 Example: Batch repair multiple tags")
    repair_dict = {
        "</bibbl>": "</bibl>",
        "<bibbl>": "<bibl>",
        "</p</bibl>": "</p></bibl>"
    }
    # toolkit.batch_repair_tags(repair_dict, scope='global')
    
    # Example 3: Column-specific repair
    print("\n9.3 Example: Column-specific repair")
    # toolkit.repair_specific_tag("bad_tag", "good_tag", 
    #                           scope='column', 
    #                           column='Principal English Translation')
    
    # Example 4: Find and review problem entries
    print("\n9.4 Example: Find problem entries")
    # problem_entries = toolkit.find_problem_entries("<Concanauhtli>")
    
    # Example 5: Re-analyze after changes
    print("\n9.5 Example: Re-analyze after manual changes")
    analysis = toolkit.reanalyze_after_changes()
    
    print("\n✅ Manual processing toolkit ready for use!")

## Usage Examples

In [42]:
# Step 9: Manual Processing Utilities  
step9_usage_examples(conn)

# Create toolkit for manual processing
toolkit = create_manual_processing_toolkit(conn)

# Example: Fix any remaining issues
# toolkit.repair_specific_tag("problematic_tag", "fixed_tag", scope='global')
# toolkit.reanalyze_after_changes()

STEP 9: MANUAL PROCESSING TOOLKIT EXAMPLES

9.1 Example: Fix specific malformed tag

9.2 Example: Batch repair multiple tags

9.3 Example: Column-specific repair

9.4 Example: Find problem entries

9.5 Example: Re-analyze after manual changes
🔍 RE-ANALYZING AFTER MANUAL CHANGES
   Using final dataset table: 08_final_complete_dataset_stage
   📊 Rows: 31,806
   📚 Citations: 53,497
   🔗 Cross-refs: 17,665

✅ Manual processing toolkit ready for use!


## DIY Data Clean-Up

In [47]:
# ==========================================
# STEP 9: COMPLETE FINAL CLEANUP
# ==========================================

# First, define the helper function
def check_remaining_non_html_tags(conn: sqlite3.Connection, 
                                 todd_tags: Dict[str, str]) -> Dict:
    """Check if Todd's manual tags still exist in our final dataset"""
    
    print("🔍 CHECKING FOR REMAINING NON-HTML TAGS")
    
    # Load your final dataset
    final_df = pd.read_sql("SELECT * FROM [08_final_complete_dataset_stage]", conn)
    
    found_tags = {}
    not_found_tags = {}
    
    for original_tag, encoded_tag in todd_tags.items():
        # Count occurrences of the original unencoded tag
        total_occurrences = 0
        locations = []
        
        for col in final_df.columns:
            for idx, cell_value in final_df[col].items():
                if pd.notna(cell_value) and original_tag in str(cell_value):
                    total_occurrences += str(cell_value).count(original_tag)
                    locations.append(f"Row {idx}, Col {col}")
        
        if total_occurrences > 0:
            found_tags[original_tag] = {
                'count': total_occurrences,
                'encoded_version': encoded_tag,
                'sample_locations': locations[:3]  # First 3 locations
            }
        else:
            not_found_tags[original_tag] = "Already handled by automation"
    
    print(f"   ✅ Tags already handled by your Step 5: {len(not_found_tags)}")
    print(f"   ⚠️  Tags still needing encoding: {len(found_tags)}")
    
    if found_tags:
        print("\n   REMAINING TAGS NEEDING MANUAL ENCODING:")
        for tag, info in list(found_tags.items())[:5]:  # Show first 5
            print(f"      '{tag}': {info['count']} occurrences")
    
    return {
        'automation_success': not_found_tags,
        'manual_needed': found_tags,
        'automation_rate': len(not_found_tags) / len(todd_tags) * 100
    }

# Then add the complete Step 9 function (from the documents you provided)
def step9_final_non_html_cleanup(conn: sqlite3.Connection) -> pd.DataFrame:
    """Step 9: Final cleanup using Todd's manual dictionary"""
    
    print("🧹 STEP 9: FINAL NON-HTML TAG CLEANUP")
    print("Using Todd's manual dictionary for edge cases...")
    
    # Todd's complete manual dictionary
    nahuatl_tags_2_reformat = {
        '<TLACOOCELUTL: [...] yoan qujtocaiotia, tlacomjztli, tepiton, pachtontli, melacpil: vel ixqujch, in castillan mjzton, nexeoac, ticeoac, cujcujltic, ocelocujcujltic, molchachapatz>': '&lt;TLACOOCELUTL: [...] yoan qujtocaiotia, tlacomjztli, tepiton, pachtontli, melacpil: vel ixqujch, in castillan mjzton, nexeoac, ticeoac, cujcujltic, ocelocujcujltic, molchachapatz&gt;',
        '<Concanauhtli>': '&lt;Concanauhtli&gt;',
        '<ue>': '&lt;ue&gt;',
        '<Canauhtli>': '&lt;Canauhtli&gt;',
        '<ZŌL-IN>': '&lt;ZŌL-IN&gt;',
        '<American White Pelican>': '&lt;American White Pelican&gt;',
        '<when feeding>': '&lt;when feeding&gt;',
        '<Ā-TŌTO-LIN>': '&lt;Ā-TŌTO-LIN&gt;',
        '<with synonym atapalcatl>': '&lt;with synonym atapalcatl&gt;',
        '<Ā-COYO-TL>': '&lt;Ā-COYO-TL&gt;',
        '<with synonyms acoyotl, atotlin>': '&lt;with synonyms acoyotl, atotlin&gt;',
        '<n>': '&lt;n&gt;',
        '<Wood Stork>': '&lt;Wood Stork&gt;',
        '<Tenitztli>': '&lt;Tenitztli&gt;',
        '<ĀCAL-LI, "boat">': '&lt;ĀCAL-LI, "boat"&gt;',
        '<XIUH-TŌTŌ-TL>': '&lt;XIUH-TŌTŌ-TL&gt;',
        '<Canyon towhee, Ilamatototl>': '&lt;Canyon towhee, Ilamatototl&gt;',
        '<Aztec Rail, COHUIX-IN>': '&lt;Aztec Rail, COHUIX-IN&gt;',
        '<shorebirds>': '&lt;shorebirds&gt;',
        '<tzoniaiauhqui, Lesser Scaup>': '&lt;tzoniaiauhqui, Lesser Scaup&gt;',
        '<Amazon>': '&lt;Amazon&gt;',
        '<zoquiazolin, literally, "mud quail">': '&lt;zoquiazolin, literally, "mud quail"&gt;',
        '<Slender-billed Grackle>': '&lt;Slender-billed Grackle&gt;',
        '<CHĪCUA-TLI>': '&lt;CHĪCUA-TLI&gt;',
        '<for CUĀUH-TLOH-TLI>': '&lt;for CUĀUH-TLOH-TLI&gt;',
        '<unlike the Peregrine>': '&lt;unlike the Peregrine&gt;',
        '<in Spanish, perhaps the Saker Falcon, Falco cherrug, a European species that most closely resembles the Prairie Falcon>': '&lt;in Spanish, perhaps the Saker Falcon, Falco cherrug, a European species that most closely resembles the Prairie Falcon&gt;',
        '<House Finch>': '&lt;House Finch&gt;',
        '<in Spanish>': '&lt;in Spanish&gt;',
        '<Prairie>': '&lt;Prairie&gt;',
        '<totolin>': '&lt;totolin&gt;',
        '<"paloma">': '&lt;"paloma"&gt;',
        '<boat-shaped>': '&lt;boat-shaped&gt;',
        '<Sandhill Crane>': '&lt;Sandhill Crane&gt;',
        '<late July>': '&lt;late July&gt;',
        '<Common Gallinule, Quachilton>': '&lt;Common Gallinule, Quachilton&gt;',
        '<tlhotli>': '&lt;tlhotli&gt;',
        '<of rivers>': '&lt;of rivers&gt;',
        '<each type of feather named separately>': '&lt;each type of feather named separately&gt;',
        '<duck>': '&lt;duck&gt;',
        '<Black-bellied Whistling-Duck>': '&lt;Black-bellied Whistling-Duck&gt;',
        '<Zoquicanauhtli, literally, "mud duck>': '&lt;Zoquicanauhtli, literally, "mud duck"&gt;',
        '<that is, salt water>': '&lt;that is, salt water&gt;',
        '<CANAUH-TLI>': '&lt;CANAUH-TLI&gt;',
        '<COZCA-CUAUH-TLI, likely the Crested Caracara>': '&lt;COZCA-CUAUH-TLI, likely the Crested Caracara&gt;',
        '<tzanatl>': '&lt;tzanatl&gt;',
        '<atole>': '&lt;atole&gt;',
        '<the Mourning Dove>': '&lt;the Mourning Dove&gt;',
        '</tlacoocelutl:>': '&lt;/tlacoocelutl:&gt;',
        '<tlacoocelutl: yoan="" qujtocaiotia="" tlacomjztli="" tepiton="" pachtontli="" melacpil:="" vel="" ixqujch="" in="" castillan="" mjzton="" nexeoac="" ticeoac="" cujcujltic="" ocelocujcujltic="" molchachapatz="">': '&lt;tlacoocelutl: yoan="" qujtocaiotia="" tlacomjztli="" tepiton="" pachtontli="" melacpil:="" vel="" ixqujch="" in="" castillan="" mjzton="" nexeoac="" ticeoac="" cujcujltic="" ocelocujcujltic="" molchachapatz=""&gt;',
        '<alo- poss- ichcatl -yoa:-l2 tla7>': '&lt;alo- poss- ichcatl -yoa:-l2 tla7&gt;',
        '<tlavitequjnj>': '&lt;tlavitequjnj&gt;',
        '<toznene>': '&lt;toznene&gt;',
        '<which see>': '&lt;which see&gt;',
        '<tzoniaiauhquj, which see>': '&lt;tzoniaiauhquj, which see&gt;'
    }
    
    # Create toolkit and apply batch repair
    toolkit = create_manual_processing_toolkit(conn)
    
    # Check what actually needs fixing
    validation_results = check_remaining_non_html_tags(conn, nahuatl_tags_2_reformat)
    
    if validation_results['manual_needed']:
        print(f"   📋 Found {len(validation_results['manual_needed'])} tags needing cleanup")
        
        # Only apply repairs for tags that actually exist
        needed_repairs = {tag: encoded for tag, encoded in nahuatl_tags_2_reformat.items() 
                         if tag in validation_results['manual_needed']}
        
        if needed_repairs:
            print(f"   🔧 Applying {len(needed_repairs)} repairs...")
            final_df = toolkit.batch_repair_tags(needed_repairs, scope='global')
            
            # Verify cleanup
            print("   🔍 Verifying cleanup...")
            post_cleanup = check_remaining_non_html_tags(conn, needed_repairs)
            print(f"   ✅ Final success rate: {post_cleanup['automation_rate']:.1f}%")
        else:
            print("   ✅ All tags already handled!")
            final_df = pd.read_sql("SELECT * FROM [08_final_complete_dataset_stage]", conn)
    else:
        print("   🎉 Perfect! All tags already encoded by automation")
        final_df = pd.read_sql("SELECT * FROM [08_final_complete_dataset_stage]", conn)
    
    # Save final dataset
    save_intermediate_stage_sqlite(final_df, "09_final_manual_cleanup", conn)
    
    # Final summary
    print("\n" + "=" * 70)
    print("🎉 COMPLETE nahuatLEX PROCESSING PIPELINE FINISHED!")
    print("=" * 70)
    print(f"📊 Step 5 Automation Success: 80%+ (excellent!)")
    print(f"🧹 Step 9 Manual Cleanup: Handles remaining edge cases")
    print(f"💾 Final dataset: 09_final_manual_cleanup_stage")
    print(f"🚀 Ready for nahuatLEX website integration!")
    print("=" * 70)
    
    return final_df

# ==========================================
# USAGE: Complete Step 9 Execution
# ==========================================

# Run the complete Step 9 cleanup
final_dataset = step9_final_non_html_cleanup(conn)

# Optional: Create toolkit for any additional manual processing needs
toolkit = create_manual_processing_toolkit(conn)

# The toolkit is available for any edge cases you discover later:
# toolkit.repair_specific_tag("problematic_tag", "fixed_tag", scope='global')
# toolkit.find_problem_entries("search_pattern")
# toolkit.reanalyze_after_changes()

🧹 STEP 9: FINAL NON-HTML TAG CLEANUP
Using Todd's manual dictionary for edge cases...
🔍 CHECKING FOR REMAINING NON-HTML TAGS
   ✅ Tags already handled by your Step 5: 35
   ⚠️  Tags still needing encoding: 20

   REMAINING TAGS NEEDING MANUAL ENCODING:
      '<American White Pelican>': 2 occurrences
      '<when feeding>': 2 occurrences
      '<with synonym atapalcatl>': 2 occurrences
      '<with synonyms acoyotl, atotlin>': 2 occurrences
      '<Wood Stork>': 1 occurrences
   📋 Found 20 tags needing cleanup
   🔧 Applying 20 repairs...
🔧 BATCH TAG REPAIR (20 repairs)
   Repairing: '<American White Pelican>' → '&lt;American White Pelican&gt;'
     ✅ 2 changes in Attestations from sources in English
   Repairing: '<when feeding>' → '&lt;when feeding&gt;'
     ✅ 2 changes in Attestations from sources in English
   Repairing: '<with synonym atapalcatl>' → '&lt;with synonym atapalcatl&gt;'
     ✅ 2 changes in Attestations from sources in English
   Repairing: '<with synonyms acoyotl, atotl

# Step 10: 